<a href="https://colab.research.google.com/github/mariemtouihri/GRAM-Metric/blob/main/Weighted_degree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Import libraries and data

In [ ]:
import numpy as np
import networkx as nx
from scipy.stats import spearmanr, kendalltau
import matplotlib.pyplot as plt
import community
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
  Load all matrices from my drive in a dictionary (There are 8 Groups of matrices,
   each time we will work with a specific group in all cells and then repeat all
    of it to the next group)

"""

# Read the adjacency matrix
loaded_data_dict = {}
noises = [i for i in range(0,110,10)]
j=4 # To select G1 and so on
for noise in noises:
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/Copy of Copie de matrices_noise_G{j}_{noise}%.npy'
  loaded_data = np.load(file_path)
  loaded_data_dict[noise] = loaded_data

print(f"Matrices G{j} are loaded successfully!")

### Define needed functions

In [ ]:
def w_degree (matrices):
  w_in = []
  w_out = []
  for i in range(len(matrices)):

    matrix = matrices[i]
    G = nx.DiGraph(matrix)

    weighted_in_degree_centrality = nx.in_degree_centrality(G)
    w_in_degree_values = list(weighted_in_degree_centrality.values())

    weighted_out_degree_centrality= nx.out_degree_centrality(G)
    w_out_degree_values = list(weighted_out_degree_centrality.values())

    w_in.extend(w_in_degree_values)
    w_out.extend(w_out_degree_values)

    w = [x + y for x, y in zip(w_in, w_out)]

  return w

In [ ]:
# This metric works on node level ! (it is different from metrics that work on graph level)
def calculate_correlations(original_values, modified_values, j, x):

  # Add noise to not get nan as correlation values
  noise = np.random.uniform(0, 1e-10, len(original_values))
  original_values += noise
  noise = np.random.uniform(0, 1e-10, len(modified_values))
  modified_values += noise

  p = []
  sp = []
  ktau = []
  corr_dict = {}
  for i in range(0, len(original_values), 90):
    # Calculate Pearson Correlation
    p_correlation = np.corrcoef(original_values[i:i+90], modified_values[i:i+90])[0, 1]
    p.append(p_correlation)

    # Calculate Spearman correlation
    sp_correlation, _ = spearmanr(original_values[i:i+90], modified_values[i:i+90])
    sp.append(sp_correlation)

    # Calculate Kendall Tau correlation
    ktau_correlation, p_value = kendalltau(original_values[i:i+90], modified_values[i:i+90])
    ktau.append(ktau_correlation)


  # Store correlation results to a json file for each matrix
  corr_dict['pearson_values'] = p
  corr_dict['spearman_values'] = sp
  corr_dict['ktau_values'] = ktau


  # Define the file path for the new JSON file
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_all_correlations_matrices_G{j}_{x}%.json'

  # Serialize and save the dictionary to the new file
  with open(file_path, 'w') as json_file:
      json.dump(corr_dict, json_file)

  print(f"Results saved to {file_path}")




  # Calculate avg of correlations of all matrices
  pearson_value = np.mean(p, axis=0)
  spearman_value = np.mean(sp, axis=0)
  ktau_value = np.mean(ktau, axis=0)

  return pearson_value, spearman_value, ktau_value

### Start calculations and save them to files


In [ ]:
"""
  Load all matrices from my drive in a dictionary (There are 8 Groups of matrices,
   each time we will work with a specific group in all cells and then repeat all
    of it to the next group)

"""

# Read the adjacency matrix
loaded_data_dict = {}
noises = [i for i in range(0,110,10)]
 # To select G1 and so on
for j in range(5,9,1):

  for noise in noises:
    file_path = f'/content/drive/MyDrive/Colab_Notebooks/Copy of Copie de matrices_noise_G{j}_{noise}%.npy'
    loaded_data = np.load(file_path)
    loaded_data_dict[noise] = loaded_data

  print(f"Matrices G{j} are loaded successfully!")





  w_degree_dict = {} # to store results in a dictionary

  for percent, loaded_data in loaded_data_dict.items():
      w_degree_dict[percent] = w_degree(loaded_data)


  # store results in an .npy file

  dict_values = list(w_degree_dict.values())
  dict_array = np.array(dict_values)

  file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_values_matrices_G{j}.npy'

  np.save(file_path, dict_array)
  print(f"Values G{j} are saved")

In [ ]:
j=1
file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_values_matrices_G{j}.npy'
data = np.load(file_path)
noises = [ i for i in range(0,110,10)]
w_degree_dict= {}
i=0
for noise in noises:
  w_degree_dict[noise] = data[i]
  i+=1

print(len(w_degree_dict))

In [ ]:
noises = [ i for i in range(0,110,10)]

for j in  range(1,9,1):
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_values_matrices_G{j}.npy'
  data = np.load(file_path)
  hub_auth_dict= {}
  i=0
  for noise in noises:
      hub_auth_dict[noise] = data[i]
      i+=1

  print(len(hub_auth_dict))


  pearson_values = [1]
  spearman_values = [1]
  ktau_values = [1]

  for i in range(10,110,10):
    pearson_value, spearman_value, ktau_value = calculate_correlations(hub_auth_dict[0], hub_auth_dict[i], j, i)
    pearson_values.append(pearson_value)
    spearman_values.append(spearman_value)
    ktau_values.append(ktau_value)



  # Store correlation results to a json file for each G

  corr_dict = {}
  corr_dict['pearson_values'] = pearson_values
  corr_dict['spearman_values'] = spearman_values
  corr_dict['ktau_values'] = ktau_values


  # Define the file path for the new JSON file
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_correlations_matrices_G{j}.json'

  # Serialize and save the dictionary to the new file
  with open(file_path, 'w') as json_file:
      json.dump(corr_dict, json_file)

  print(f"Results saved to {file_path}")


In [ ]:
pearson_values = [1]
spearman_values = [1]
ktau_values = [1]

for i in range(0,110,10):
  pearson_value, spearman_value, ktau_value = calculate_correlations(w_degree_dict[0], w_degree_dict[i])
  pearson_values.append(pearson_value)
  spearman_values.append(spearman_value)
  ktau_values.append(ktau_value)
  # print(f"for {i}% noise (Pearson value: {pearson_value}), (Spearman value: {spearman_value}), (Kendall-tau value: {ktau_value})")



# Store correlation results to a json file for each G

corr_dict = {}
corr_dict['pearson_values'] = pearson_values
corr_dict['spearman_values'] = spearman_values
corr_dict['ktau_values'] = ktau_values


# Define the file path for the new JSON file
file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_correlations_matrices_G{j}.json'

# Serialize and save the dictionary to the new file
with open(file_path, 'w') as json_file:
    json.dump(corr_dict, json_file)

print(f"Results saved to {file_path}")



In [ ]:
# read correlations from stored files G1, G2, ..

pearson_values = []
spearman_values = []
ktau_values = []


for j in range(8):

  # Define the path to your JSON file
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_correlations_matrices_G{j+1}.json'

  # Open and read the JSON file
  with open(file_path, 'r') as json_file:
      corr_dict = json.load(json_file)

      if j== 0:
        pearson_values.extend(corr_dict["pearson_values"])
        spearman_values.extend(corr_dict["spearman_values"])
        ktau_values.extend(corr_dict["ktau_values"])
      else:
        pearson_values = np.vstack((pearson_values, corr_dict["pearson_values"]))
        spearman_values = np.vstack((spearman_values, corr_dict["spearman_values"]))
        ktau_values = np.vstack((ktau_values, corr_dict["ktau_values"]))


# Ready to be plotted (results from all matrices horray!)
pearson_values = list(np.mean(pearson_values, axis=0))
spearman_values = list(np.mean(spearman_values, axis=0))
ktau_values = list(np.mean(ktau_values, axis=0))


In [ ]:
# Store correlation results to a json file for each G
corr_dict = {}
corr_dict['pearson_values'] = pearson_values
corr_dict['spearman_values'] = spearman_values
corr_dict['ktau_values'] = ktau_values

# Define the file path for the new JSON file
file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_correlations_all_matrices.json'

# Serialize and save the dictionary to the new file
with open(file_path, 'w') as json_file:
    json.dump(corr_dict, json_file)

print(f"Results saved to {file_path}")

In [ ]:
# Plot results
modification_rates = [f"{i}%" for i in range(0,100,10)]
plt.figure(figsize=(8, 6))

plt.plot(modification_rates, pearson_values, marker='o', label= "Pearson")
plt.plot(modification_rates, spearman_values, marker='o', label= "Spearman")
plt.plot(modification_rates, ktau_values, marker='o', label= "Kendall Tau")

plt.xlabel("Modification Percentage")
plt.ylabel("Correlation")
plt.title("Correlation of Weighted Degree Centrality (In-degree and Out_degree) (Original vs Modified)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot results
modification_rates = [f"{i}%" for i in range(0,100,10)]
plt.figure(figsize=(8, 6))

plt.plot(modification_rates, pearson_values, marker='o', label= "Pearson")
plt.plot(modification_rates, spearman_values, marker='o', label= "Spearman")
plt.plot(modification_rates, ktau_values, marker='o', label= "Kendall Tau")

plt.xlabel("Modification Percentage")
plt.ylabel("Correlation")
plt.title("Correlation of Weighted Degree Centrality (In-degree and Out_degree) (Original vs Modified)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# read correlations from stored files G1, G2, ..

pearson_values = []
spearman_values = []
ktau_values = []


for j in range(8):

  # Define the path to your JSON file
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/w_degree/w_degree_correlations_matrices_G{j+1}.json'

  # Open and read the JSON file
  with open(file_path, 'r') as json_file:
      corr_dict = json.load(json_file)

      if j== 0:
        pearson_values.extend(corr_dict["pearson_values"])
        spearman_values.extend(corr_dict["spearman_values"])
        ktau_values.extend(corr_dict["ktau_values"])
      else:
        pearson_values = np.vstack((pearson_values, corr_dict["pearson_values"]))
        spearman_values = np.vstack((spearman_values, corr_dict["spearman_values"]))
        ktau_values = np.vstack((ktau_values, corr_dict["ktau_values"]))


# Ready to be plotted (results from all matrices horray!)
pearson_values = list(np.mean(pearson_values, axis=0))
spearman_values = list(np.mean(spearman_values, axis=0))
ktau_values = list(np.mean(ktau_values, axis=0))


In [ ]:
# Plot results
modification_rates = [f"{i}%" for i in range(0,110,10)]
plt.figure(figsize=(8, 6))

plt.plot(modification_rates, pearson_values, marker='o', label= "Pearson")
plt.plot(modification_rates, spearman_values, marker='o', label= "Spearman")
plt.plot(modification_rates, ktau_values, marker='o', label= "Kendall Tau")

plt.xlabel("Modification Percentage")
plt.ylabel("Correlation")
plt.title("Correlation of Weighted Degree Centrality (In-degree and Out_degree) (Original vs Modified)")
plt.legend()
plt.grid(True)
plt.show()